In [2]:
from pathlib import Path
import numpy as np
import pandas as pd

In [3]:
# En caso de que se ejecute clonando el repositorio

DATA_DIR = Path.cwd().resolve().parent / "datos"

datos_titanic = pd.read_parquet(DATA_DIR / "02_datos_con_tipo_de_dato_ajustado_titanic.parquet", engine="pyarrow")

In [4]:
columnas_seleccionadas = [
    "pclass",
    "sex",
    "age",
    "sibsp",
    "parch",
    "fare",
    "embarked",
    "survived",
]

In [5]:
df_titanic = datos_titanic[columnas_seleccionadas]

df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   pclass    1309 non-null   int64   
 1   sex       1309 non-null   category
 2   age       1046 non-null   float64 
 3   sibsp     1309 non-null   int8    
 4   parch     1309 non-null   int8    
 5   fare      1308 non-null   float64 
 6   embarked  1307 non-null   category
 7   survived  1309 non-null   bool    
dtypes: bool(1), category(2), float64(2), int64(1), int8(2)
memory usage: 37.5 KB


# Ajuste de tipos y eliminación de duplicados

In [6]:
df_titanic["survived"] = df_titanic["survived"].astype(bool).astype(int)

/var/folders/l4/1t58ff412ss3dr4zyn2_6wwhch78n1/T/ipykernel_7015/2103773122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_titanic["survived"] = df_titanic["survived"].astype(bool).astype(int)


In [7]:
df_titanic = df_titanic.drop_duplicates()

## Definición de Variables Numéricas, Categóricas y Categóricas Ordinales

In [8]:
cols_numeric = ["age", "fare", "sibsp", "parch"]
cols_categoric = ["sex", "embarked"]
cols_categoric_ord = ["pclass"]

## Importación de Herramientas de Preprocesamiento

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

* `ColumnTransformer:` Permite aplicar diferentes transformaciones a distintas columnas de un DataFrame.

* `SimpleImputer:` Se encarga de imputar valores faltantes (por ejemplo, usar la mediana para columnas numéricas).

* `Pipeline:` Forma de encadenar transformaciones y modelado en un solo objeto.

* `OneHotEncoder:` Transforma columnas categóricas en múltiples columnas binarias (codificación One-Hot).

* `OrdinalEncoder:` Asigna valores numéricos ordenados a cada categoría.

## Definición de Pipelines

En este ejemplo, crearemos **dos pipelines de preprocesamiento** para evaluar el impacto de dos enfoques diferentes en las métricas del modelo. Los enfoques son:

1. **Enfoque 1: Imputación con la Mediana (sin Escalado)**  
   - Se imputan las variables numéricas usando la mediana.
   - No se aplica escalado.

2. **Enfoque 2: Imputación con la Media + Escalado**  
   - Se imputan las variables numéricas usando la media.
   - Se aplica el escalado estándar (StandardScaler).

In [10]:
numeric_pipe_median = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)


numeric_pipe_mean_scale = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)


categorical_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder()),
    ]
)


categorical_ord_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OrdinalEncoder()),
    ]
)

## Creación del Transformer Completo

* `transformers:` Lista de tuplas indicando (nombre_del_transformador, objeto_de_transformación, columnas_a_transformar).

* `preprocessor:` Objeto que, al aplicarse a los datos, ejecuta las transformaciones definidas en los pipelines para cada grupo de columnas.


**Tip:** Recordar que la fórmula de la estandarización usada en `StandardScaler` es:

$$
X_{\text{scaled}} = \frac{X - \mu}{\sigma}
$$

Donde:
- \(X\): Valor original.
- (\mu\): Media de la característica.
- \(\sigma\): Desviación estándar de la característica.

In [11]:
preprocessor_median = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipe_median, cols_numeric),
        ("categoric", categorical_pipe, cols_categoric),
        ("categoric ordinal", categorical_ord_pipe, cols_categoric_ord),
    ]
)

preprocessor_median

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['sex', 'embarked']),
                                ('categoric ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OrdinalEncoder())]),
                                 ['pclass'])])

In [12]:
preprocessor_mean_scale = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipe_mean_scale, cols_numeric),
        ("categoric", categorical_pipe, cols_categoric),
        ("categoric ordinal", categorical_ord_pipe, cols_categoric_ord),
    ]
)

preprocessor_mean_scale

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['sex', 'embarked']),
                                ('categoric ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OrdinalEncoder())]),
                                 ['pclass'])])

# Dividir en train/test

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_features = df_titanic.drop("survived", axis="columns")
Y_target = df_titanic["survived"]

# 80% train, 20% test

x_train, x_test, y_train, y_test = train_test_split(
    X_features, Y_target, test_size=0.2, stratify=Y_target
)

print("Dimensiones del dataset de entrenamiento", x_train.shape, y_train.shape)

print("Dimensiones del dataset de validación", x_test.shape, y_test.shape)

Dimensiones del dataset de entrenamiento (891, 7) (891,)
Dimensiones del dataset de validación (223, 7) (223,)


#### Solo si quiero ver cómo quedan los datos después de aplicar el preprocesador

In [15]:
# train pipeline
preprocessor_mean_scale.fit(x_test)

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['sex', 'embarked']),
                                ('categoric ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OrdinalEncoder())]),
                                 ['pclass'])])

In [16]:
# obtener el nombre de las columnas de salida del preprocesamiento
# usando .get_feature_names_out()
feature_names = preprocessor_mean_scale.get_feature_names_out()

feature_names

array(['numeric__age', 'numeric__fare', 'numeric__sibsp',
       'numeric__parch', 'categoric__sex_female', 'categoric__sex_male',
       'categoric__embarked_C', 'categoric__embarked_Q',
       'categoric__embarked_S', 'categoric ordinal__pclass'], dtype=object)

In [17]:
# transform x_Test with preprocessor and pandas output set
x_test_transformed = preprocessor_mean_scale.transform(x_test)

x_test_transformed = pd.DataFrame(x_test_transformed, columns=feature_names)

x_test_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   numeric__age               223 non-null    float64
 1   numeric__fare              223 non-null    float64
 2   numeric__sibsp             223 non-null    float64
 3   numeric__parch             223 non-null    float64
 4   categoric__sex_female      223 non-null    float64
 5   categoric__sex_male        223 non-null    float64
 6   categoric__embarked_C      223 non-null    float64
 7   categoric__embarked_Q      223 non-null    float64
 8   categoric__embarked_S      223 non-null    float64
 9   categoric ordinal__pclass  223 non-null    float64
dtypes: float64(10)
memory usage: 17.6 KB


In [18]:
x_test_transformed.head(3)

,numeric__age,numeric__fare,numeric__sibsp,numeric__parch,categoric__sex_female,categoric__sex_male,categoric__embarked_C,categoric__embarked_Q,categoric__embarked_S,categoric ordinal__pclass
0,-3.203214e-01,-0.447255,-0.563611,-0.461898,1.0,0.0,0.0,0.0,1.0,2.0
1,1.138672e+00,-0.122217,-0.563611,1.428069,1.0,0.0,0.0,0.0,1.0,1.0
2,-2.728098e-16,-0.339461,0.589464,0.483086,0.0,1.0,1.0,0.0,0.0,2.0


# Modelos

Cabe resaltar que las buenas prácticas de Python indican que todas las importaciones se deben realizar en la parte superior del notebook, sin embargo, para efectos de este ejercicio, en cada sección hemos importado lo que necesitamos.

En este caso solo probaremos dos modelos:

* Logistic Regressión / Regresion Logística

* Random Forest / Bosques Aleatorios

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

### Selección de modelos básicos

Para este caso, ya que solo se probarán dos, una vez determinemos el que mejor desempeño tuvo será necesario realizar el "finetune" sobre el elegido.

Por esto, se entrenarán con los hiperparámetros "por defecto".

In [20]:
# Función "helping" para tener métricas concretas luego de entrenar el modelo

def resumen_clasificación(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)

    return {"exactitud": acc,
            "precision": prec,
            "recall": recall,
            "f1": f1,
            "roc": roc}


In [21]:
# Aunque ya las teníamos definidas unas celdas arriba, las volvemos a escribir por facilidad en la lectura del código:

cols_numeric = ["age", "fare", "sibsp", "parch"]
cols_categoric = ["sex", "embarked"]
cols_categoric_ord = ["pclass"]

In [22]:
# Definir los modelos a probar

modelos = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42)
}

Recordar que los preprocesadores se llaman:
* `preprocessor_median`
* `preprocessor_mean_scale`

In [23]:
print(modelos)
print("----")
print(type(modelos))

{'RandomForest': RandomForestClassifier(random_state=42), 'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42)}
----
<class 'dict'>


In [24]:
# Crear pipelines completos para cada combinación de modelo y preprocesador
pipelines = {} #Nótese que es un diccionario

for modelo_nombre, modelo in modelos.items():
    # Pipeline con imputación mediana (sin escalado)
    pipelines[f"{modelo_nombre}_median"] = Pipeline([
        ("preprocessing", preprocessor_median),
        ("classifier", modelo)
    ])
    # Pipeline con imputación con media + escalado
    pipelines[f"{modelo_nombre}_mean_scale"] = Pipeline([
        ("preprocessing", preprocessor_mean_scale),
        ("classifier", modelo)
    ])

In [25]:
pipelines

{'RandomForest_median': Pipeline(steps=[('preprocessing',
                  ColumnTransformer(transformers=[('numeric',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='median'))]),
                                                   ['age', 'fare', 'sibsp',
                                                    'parch']),
                                                  ('categoric',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='most_frequent')),
                                                                   ('onehot',
                                                                    OneHotEncoder())]),
                                                   ['sex', 'embarked']),
                                                  ('categoric ord

### Entrenar y evaluar cada pipeline

In [26]:
# Entrenar y evaluar cada pipeline
resultados = {}

for nombre_pipeline, pipeline in pipelines.items():
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    resultados[nombre_pipeline] = resumen_clasificación(y_test, y_pred)

In [27]:
import pprint
print("Resumen de Métricas para Cada Pipeline:")
pprint.pprint(resultados)

Resumen de Métricas para Cada Pipeline:
{'LogisticRegression_mean_scale': {'exactitud': 0.7443946188340808,
                                   'f1': 0.6885245901639344,
                                   'precision': 0.7,
                                   'recall': 0.6774193548387096,
                                   'roc': 0.734863523573201},
 'LogisticRegression_median': {'exactitud': 0.7354260089686099,
                               'f1': 0.6810810810810811,
                               'precision': 0.6847826086956522,
                               'recall': 0.6774193548387096,
                               'roc': 0.7271712158808933},
 'RandomForest_mean_scale': {'exactitud': 0.726457399103139,
                             'f1': 0.6432748538011696,
                             'precision': 0.7051282051282052,
                             'recall': 0.5913978494623656,
                             'roc': 0.7072373862696443},
 'RandomForest_median': {'exactitud': 0.748878923766

La forma de ver los resultados con `pretty print` no es la más clara. Pero recuerda que `resultados` es un diccionario. Por lo tanto, como ya eres experto en Pandas, convertirlo en un DataFrame está a una línea de código:

In [28]:
# 1. Convertir el diccionario 'resultados' en un DataFrame
df_resultados = pd.DataFrame(resultados).T

# 2. Ordenar por la columna 'exactitud' de manera descendente
df_resultados_sorted = df_resultados.sort_values(by="exactitud", ascending=False)

# 3. Visualizar el DataFrame resultante
df_resultados_sorted

,exactitud,precision,recall,f1,roc
RandomForest_median,0.748879,0.746667,0.602151,0.666667,0.727998
LogisticRegression_mean_scale,0.744395,0.700000,0.677419,0.688525,0.734864
LogisticRegression_median,0.735426,0.684783,0.677419,0.681081,0.727171
RandomForest_mean_scale,0.726457,0.705128,0.591398,0.643275,0.707237


# Validación Cruzada con 5 Folds y Visualización de Resultados con Boxplot (Plotly)

La **validación cruzada** (cross-validation) es una técnica que consiste en dividir los datos de entrenamiento en varios **folds** (segmentos). En cada iteración, se entrena el modelo con todos los folds menos uno, y se evalúa en el fold restante. Al rotar por todos los folds, se obtiene una evaluación más robusta y menos dependiente de una sola partición de los datos.

En este ejemplo, usaremos:
- **5 folds** de validación cruzada.
- Cuatro pipelines distintos que combinan distintos preprocesamientos y modelos.
- **Plotly** para crear un boxplot con los valores de *accuracy* en cada fold.

> **¿Por qué es útil la validación cruzada?**  
> - **Mayor robustez** en la evaluación: se reduce la varianza asociada a la división de los datos.
> - **Mejor uso de los datos**: todos los ejemplos pasan por el proceso de validación en algún fold.
> - **Detección de sobreajuste**: si el modelo funciona muy bien en algunos folds y mal en otros, se hace evidente la inestabilidad o sobreajuste.

In [29]:
from sklearn.model_selection import cross_val_score

In [30]:
# DataFrame para guardar los resultados de cada fold
df_cv_results = pd.DataFrame(columns=["pipeline", "fold", "accuracy"])

# Número de folds
cv_folds = 5

A continuación:

* Se construye un pequeño DataFrame con tantas filas como folds (`cv_folds`), en este caso 5.

* `"pipeline"`: `[pipeline_name]*cv_folds`: Crea una columna "pipeline" con el mismo valor pipeline_name repetido `cv_folds` veces, para etiquetar a qué modelo pertenecen los valores de accuracy.

* `"fold": list(range(1, cv_folds+1)):` Crea la columna "fold" con valores [1, 2, ..., 5] para identificar el número de fold.

* "accuracy": scores: Inserta los valores de accuracy devueltos por `cross_val_score`.

In [31]:
pipelines

{'RandomForest_median': Pipeline(steps=[('preprocessing',
                  ColumnTransformer(transformers=[('numeric',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='median'))]),
                                                   ['age', 'fare', 'sibsp',
                                                    'parch']),
                                                  ('categoric',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='most_frequent')),
                                                                   ('onehot',
                                                                    OneHotEncoder())]),
                                                   ['sex', 'embarked']),
                                                  ('categoric ord

In [32]:
for pipeline_name, pipeline_obj in pipelines.items():
    # cross_val_score entrena y evalúa en 5 folds, devolviendo un array con 5 valores de accuracy
    scores = cross_val_score(pipeline_obj, x_train, y_train, cv=cv_folds, scoring="accuracy")

    # Crear un DataFrame temporal con la información de cada fold
    temp_df = pd.DataFrame({
        "pipeline": [pipeline_name]*cv_folds,
        "fold": list(range(1, cv_folds+1)),
        "accuracy": scores
    })

    # Concatenar al DataFrame global
    df_cv_results = pd.concat([df_cv_results, temp_df], ignore_index=True)

/var/folders/l4/1t58ff412ss3dr4zyn2_6wwhch78n1/T/ipykernel_7015/2390259851.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cv_results = pd.concat([df_cv_results, temp_df], ignore_index=True)


In [33]:
# Observa los resultados
df_cv_results.head(20)

,pipeline,fold,accuracy
0,RandomForest_median,1,0.709497
1,RandomForest_median,2,0.758427
2,RandomForest_median,3,0.797753
3,RandomForest_median,4,0.775281
4,RandomForest_median,5,0.780899
5,RandomForest_mean_scale,1,0.703911
6,RandomForest_mean_scale,2,0.752809
7,RandomForest_mean_scale,3,0.786517
8,RandomForest_mean_scale,4,0.769663
9,RandomForest_mean_scale,5,0.792135


In [34]:
import plotly.express as px

fig = px.box(
    df_cv_results,
    x="pipeline",
    y="accuracy",
    points="all",  # Muestra los puntos individuales además del boxplot
    title="Distribución de Accuracy por Pipeline (5-Fold CV)"
)

fig.show()

/Users/david.palacio/Documents/academia/data-projects-lab/.venv/lib/python3.11/site-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




# Elección del modelo final

* `RandomForest_median`


Aunque las métricas pueden indicar que otro pipeline (por ejemplo, uno basado en escalado e imputación con la media) mostró resultados ligeramente superiores, existen varias razones pedagógicas y prácticas para escoger el modelo **RandomForest_median**:

1. **Simplicidad del Preprocesamiento:**
   - **Imputación con Mediana:**  
     Utilizar la mediana es una técnica robusta frente a valores extremos, lo que ayuda a los estudiantes a comprender cómo se pueden manejar los outliers sin la complejidad del escalado.  
   - **Claridad en la Transformación:**  
     Al no incluir el escalado, se reduce la cantidad de transformaciones, permitiendo que los alumnos se centren en entender el efecto de la imputación.

2. **Robustez y Consistencia del Modelo:**
   - **RandomForest:**  
     Este modelo es conocido por ser robusto y menos sensible a cambios menores en los datos, lo cual lo hace ideal para propósitos de enseñanza.  
   - **Estabilidad en la Validación Cruzada:**  
     Aunque otro enfoque pueda tener una métrica ligeramente mejor, RandomForest_median demuestra un comportamiento consistente a lo largo de diferentes folds, lo que es un buen ejemplo para discutir la estabilidad del modelo.

3. **Facilidad de Interpretación:**
   - **Interpretación de Resultados:**  
     Random Forest ofrece la posibilidad de analizar la importancia de las variables, facilitando la discusión sobre cómo diferentes características contribuyen a la predicción.  
   - **Menor Complejidad en el Pipeline:**  
     Sin la adición del escalado, el pipeline es más sencillo y fácil de explicar, lo que puede ser beneficioso para estudiantes que están comenzando en machine learning.

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [36]:
# Creamos el pipeline que integra el preprocesador (preprocessor_median) y el clasificador
pipeline_rf_median = Pipeline([
    ("preprocessing", preprocessor_median),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [37]:
# Definimos el grid de hiperparámetros a explorar
param_grid = {
    "classifier__n_estimators": [50, 100, 200],    # Número de árboles
    "classifier__max_depth": [None, 5, 10],          # Profundidad máxima del árbol (None para sin límite)
    "classifier__min_samples_split": [2, 5, 10]      # Número mínimo de muestras para dividir un nodo
}

In [38]:
# Configuramos GridSearchCV para evaluar con 5 folds y usando la métrica "accuracy"
grid_search = GridSearchCV(
    pipeline_rf_median,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1  # Utiliza todos los cores disponibles
)

In [39]:
# Ejecutamos el grid search usando los datos de entrenamiento
grid_search.fit(x_train, y_train)

# Mostramos los mejores parámetros y el mejor accuracy obtenido en validación cruzada
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor accuracy (CV):", grid_search.best_score_)

Mejores parámetros: {'classifier__max_depth': None, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 50}
Mejor accuracy (CV): 0.8036281463812692


# Guardado del modelo en formato `joblib`

In [40]:
DATA_DIR = Path.cwd() / "Modelos"

In [41]:
import joblib

# Suponiendo que 'grid_search' es el objeto de GridSearchCV que usamos para el fine-tuning
best_model = grid_search.best_estimator_

# Imprimir el mejor modelo
print("El mejor modelo es:")
print(best_model)

# Guardar el modelo usando joblib con el nombre especificado
joblib.dump(best_model, DATA_DIR / "titanic_classification-random_forest-v1.joblib")

print("Modelo guardado como 'titanic_classification-random_forest-v1.joblib'")

El mejor modelo es:
Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fare', 'sibsp',
                                                   'parch']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'embarked']),
                                                 ('categoric ordinal',
         

## ¿Por qué guardarlo en `pickle` o `joblib`?

Guardar un modelo en formato **pickle** o **joblib** permite almacenar el estado completo del modelo entrenado (incluyendo parámetros, pesos, y la estructura del pipeline) en un archivo. Esto tiene varias ventajas:

- **Persistencia:**  
  Puedes guardar el modelo una vez entrenado y cargarlo posteriormente sin tener que volver a entrenarlo, lo que ahorra tiempo y recursos.

- **Despliegue:**  
  Facilita la implementación del modelo en producción o su uso en aplicaciones, ya que el modelo se carga rápidamente desde el archivo.

- **Compatibilidad:**  
  Joblib está optimizado para objetos que contienen grandes arreglos NumPy, lo que lo hace especialmente útil para modelos de scikit-learn, aunque pickle también es una opción válida.

La elección entre pickle y joblib depende principalmente del tamaño y complejidad del modelo:

- **pickle:** Es parte de la biblioteca estándar de Python y funciona bien para modelos pequeños o moderados.
- **joblib:** Es preferible para modelos grandes o que manejen grandes volúmenes de datos, ya que maneja la serialización de arrays de forma más eficiente.

En este ejemplo, se utilizó `joblib.dump()` para guardar el modelo en el archivo `titanic_classification-random_forest-v1.joblib`, y `joblib.load()` para cargarlo posteriormente y realizar predicciones.


# Carga del modelo y uso en datos no vistos

In [42]:
# Cargar el modelo previamente guardado
loaded_model = joblib.load(DATA_DIR / "titanic_classification-random_forest-v1.joblib")
print("Modelo cargado exitosamente.")

Modelo cargado exitosamente.


## Probar el modelo en datos no vistos

In [43]:
# Número de muestras sintéticas
np.random.seed(57)

n_samples = 50

# Generar datos sintéticos siguiendo la estructura del dataset Titanic
df_synthetic_test = pd.DataFrame({
    "pclass": np.random.choice([1, 2, 3], size=n_samples),
    "sex": np.random.choice(["male", "female"], size=n_samples),
    "age": np.random.uniform(0, 80, size=n_samples),
    "sibsp": np.random.randint(0, 4, size=n_samples),
    "parch": np.random.randint(0, 4, size=n_samples),
    "fare": np.random.uniform(10, 100, size=n_samples),
    "embarked": np.random.choice(["C", "Q", "S"], size=n_samples)
})

# Mostrar los primeros registros para verificar
df_synthetic_test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,29.549676,2,2,87.460519,C
1,2,male,22.782733,2,1,81.690288,S
2,3,male,74.169372,1,2,51.914214,Q
3,1,male,62.041734,3,2,74.020331,Q
4,3,male,51.305236,0,0,26.458023,S


## Predicciones

In [44]:
# Realizar predicciones con el modelo cargado
y_pred_synthetic = loaded_model.predict(df_synthetic_test)

print("Predicciones en datos sintéticos:")
print(y_pred_synthetic)

Predicciones en datos sintéticos:
[0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1
 1 1 0 1 0 0 1 1 0 1 1 0 0]


In [45]:
# Crear un DataFrame que incluya los datos originales y las predicciones
df_predictions = df_synthetic_test.copy()
df_predictions["prediction"] = y_pred_synthetic

print("Predicciones en datos sintéticos:")
df_predictions.head()

Predicciones en datos sintéticos:


,pclass,sex,age,sibsp,parch,fare,embarked,prediction
0,3,male,29.549676,2,2,87.460519,C,0
1,2,male,22.782733,2,1,81.690288,S,0
2,3,male,74.169372,1,2,51.914214,Q,0
3,1,male,62.041734,3,2,74.020331,Q,0
4,3,male,51.305236,0,0,26.458023,S,0


# Referencias

* https://pycaret.gitbook.io/docs#classification

* https://medium.com/@roshmitadey/pycaret-for-autom-a7b5c36a5772
